# Replication of code from Wang and Avillach (2021)

In [439]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow import keras
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Conv1D, Dropout, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, roc_auc_score
from keras import regularizers
import keras
import os
import tempfile
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.utils import shuffle
from random import seed
import plotly.express as px

In [371]:
# Downloading the 100 top SNPs from Wang & Avillach. 
# These SNPs were not provided as supplementary material but were found in Wang's github
snp_matrix = pd.read_csv('RepeatPaper/prosib_topSNPs_pheno.raw', sep=' ')

Replicating their code. Below cell is what was done in Wang's github.
<br> *Comments highlight why it doesn't work as expected*

In [419]:
random.seed(10)
only_snps = snp_matrix.drop(columns=['FID', 'IID', 'PAT', 'MAT', 'SEX'])
only_snps = shuffle(only_snps) # indices are no longer equivalent to labels
train_index = random.sample(range(only_snps.shape[0]), int(only_snps.shape[0]*0.80))
X_train = only_snps.iloc[train_index] # this line goes by index
X_test = only_snps.drop(train_index) # this line goes by label (because of shuffle this isn't the same)


y_train = X_train['PHENOTYPE'] - 1
y_test = X_test['PHENOTYPE'] - 1
X_train = X_train.drop(columns=['PHENOTYPE'])
X_test = X_test.drop(columns=['PHENOTYPE'])
# this is not copied from Wang and Avillach but is required to replicate vcfgt function (see further down in notebook)
X_train = X_train.replace(0,1) 
X_train = X_train.fillna(0)
X_test= X_test.replace(0,1)
X_test = X_test.fillna(0)

Below cell shows how it could be done correctly. *Note: running both the below cell after the above cell will replace variables due to identical variable names. Purely for explanatory purposes*

In [437]:
# Fixed code. This shows an example of how it could be done that works
snp_y = snp_matrix['PHENOTYPE'] - 1
only_snps = snp_matrix.drop(columns=['FID', 'IID', 'PAT', 'MAT', 'PHENOTYPE', 'SEX'])
only_snps = only_snps.fillna(0)
X_train, X_test, y_train, y_test = train_test_split(only_snps, snp_y, test_size=0.2, random_state=10)

In [411]:
random.seed(10)
# Just train split error so correcting for SNP coding
only_snps = snp_matrix.drop(columns=['FID', 'IID', 'PAT', 'MAT', 'SEX'])
only_snps = shuffle(only_snps) # indices are no longer equivalent to labels
train_index = random.sample(range(only_snps.shape[0]), int(only_snps.shape[0]*0.80))
X_train = only_snps.iloc[train_index] # this line goes by index
X_test = only_snps.drop(train_index) # this line goes by label (because of shuffle this isn't the same)

y_train = X_train['PHENOTYPE'] - 1
y_test = X_test['PHENOTYPE'] - 1
X_train = X_train.drop(columns=['PHENOTYPE'])
X_test = X_test.drop(columns=['PHENOTYPE'])
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

**Below is the CNN from Wang & Avillach.** *Note: the current output displayed in this notebook is from trying to replicate the code exactly (see cell '[419]' further up)*

In [420]:
X_train = X_train.to_numpy().reshape((X_train.shape[0],X_train.shape[1],1))
X_test = X_test.to_numpy().reshape((X_test.shape[0],X_test.shape[1],1))
input_shape = X_test.shape[1:]

## Actually from their paper
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Conv1D, Dropout, MaxPooling1D

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(rate = 0.5))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(rate = 0.5))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
sgd = SGD(learning_rate=0.01, momentum=0.08)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [421]:
history = model.fit(X_train,y_train, batch_size=128, epochs=95)

Epoch 1/95
25/25 [==============================] - 2s 13ms/step - loss: 0.6819 - accuracy: 0.5618
Epoch 2/95
25/25 [==============================] - 0s 13ms/step - loss: 0.6718 - accuracy: 0.5740
Epoch 3/95
25/25 [==============================] - 0s 12ms/step - loss: 0.6670 - accuracy: 0.5766
Epoch 4/95
25/25 [==============================] - 0s 12ms/step - loss: 0.6628 - accuracy: 0.5990
Epoch 5/95
25/25 [==============================] - 0s 13ms/step - loss: 0.6489 - accuracy: 0.6240
Epoch 6/95
25/25 [==============================] - 0s 12ms/step - loss: 0.6376 - accuracy: 0.6672
Epoch 7/95
25/25 [==============================] - 0s 12ms/step - loss: 0.6239 - accuracy: 0.6775
Epoch 8/95
25/25 [==============================] - 0s 12ms/step - loss: 0.6290 - accuracy: 0.6855
Epoch 9/95
25/25 [==============================] - 0s 12ms/step - loss: 0.6210 - accuracy: 0.6845
Epoch 10/95
25/25 [==============================] - 0s 12ms/step - loss: 0.6174 - accuracy: 0.6877
Epoch 11/

25/25 [==============================] - 0s 12ms/step - loss: 0.4748 - accuracy: 0.7646
Epoch 84/95
25/25 [==============================] - 0s 13ms/step - loss: 0.4661 - accuracy: 0.7816
Epoch 85/95
25/25 [==============================] - 0s 12ms/step - loss: 0.4646 - accuracy: 0.7799
Epoch 86/95
25/25 [==============================] - 0s 12ms/step - loss: 0.4717 - accuracy: 0.7729
Epoch 87/95
25/25 [==============================] - 0s 13ms/step - loss: 0.4561 - accuracy: 0.7924
Epoch 88/95
25/25 [==============================] - 0s 13ms/step - loss: 0.4636 - accuracy: 0.7732
Epoch 89/95
25/25 [==============================] - 0s 12ms/step - loss: 0.4471 - accuracy: 0.7902
Epoch 90/95
25/25 [==============================] - 0s 12ms/step - loss: 0.4528 - accuracy: 0.7934
Epoch 91/95
25/25 [==============================] - 0s 12ms/step - loss: 0.4524 - accuracy: 0.7899
Epoch 92/95
25/25 [==============================] - 0s 12ms/step - loss: 0.4228 - accuracy: 0.8056
Epoch 93/95


In [422]:
# Evaluating the mdoel gave a loss of 0.44 and an accuracy of 0.81 on the test data 
# (when using their code). Discrepency due to lack of random seed for reproducibility.
# This should raise flags as above shows an almost identical accuracy on their training data
model.evaluate(X_test,y_test)

25/25 [==============================] - 0s 4ms/step - loss: 0.4408 - accuracy: 0.8054


[0.4407598376274109, 0.805377721786499]

In [423]:
# This section wasn't included included in Wang & Avillach code but shows different metrics.
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
f1 = f1_score(y_test,y_pred)
print('accuracy:',accuracy,'\nprecision:',precision, '\nrecall:',recall,'\nf1 score:',f1)

25/25 [==============================] - 0s 4ms/step
accuracy: 0.8053777208706786 
precision: 0.811965811965812 
recall: 0.8558558558558559 
f1 score: 0.8333333333333335


In [424]:
# AUC printed (0.88) is also from Wang & Avillach code (inc two errors)
y_pred_dl = model.predict(X_test).ravel()
fpr_dl, tpr_dl, thresholds_dl = roc_curve(y_test, y_pred_dl)
auc_dl = auc(fpr_dl, tpr_dl)
print('DL Prediction AUC: %1.2f' % (auc_dl))

25/25 [==============================] - 0s 3ms/step
DL Prediction AUC: 0.88


**Generation of Figure 2 Histogram in letter to the editor.** <br> 100 simulations show that on average 80% of the training data is in the test data using their sampling method 

In [427]:
percent = []
for i in range(100):
    only_snps = snp_matrix.drop(columns=['FID', 'IID', 'PAT', 'MAT'])
    only_snps = shuffle(only_snps)
    train_index = random.sample(range(only_snps.shape[0]), int(only_snps.shape[0]*0.80))
    X_train = only_snps.iloc[train_index]
    X_test = only_snps.drop(train_index)
    overlap = len(set(X_train.index) & set(X_test.index))
    percent.append(overlap/len(X_test))
print(np.mean(percent))
print(np.std(percent))

0.7995006402048656
0.012133657416260081


In [436]:
fig = px.histogram(percent, nbins=10, labels={'value': 'Proportion of test dataset in training dataset'})
fig.show()

**Example of how Wang & Avillach's code misasigns genotypes. Function vcfgt copied from their github.** This makes up figure 1 in letter to editor

In [396]:
data = {'genotype': ['0/0', '0/1', '1/1', '0/0', '1/0']}
df = pd.DataFrame(data)

def vcfgt(x):
    output = 1
    flag = False
    x = str(x).split('/')
    if int(x[0]) > 0 and int(x[1]) > 0:
        flag = True 
    if flag:
        output = 2
    return output

# Apply the function to the 'genotype' column
df['vcfgt_result'] = df['genotype'].apply(vcfgt)
df = pd.concat([df,pd.DataFrame({'genotype': [np.nan], 'vcfgt_result': [0]})])
df['expected_result'] = [0,1,2,0,1,0]
print(df)

  genotype  vcfgt_result  expected_result
0      0/0             1                0
1      0/1             1                1
2      1/1             2                2
3      0/0             1                0
4      1/0             1                1
0      NaN             0                0


**100 Simulations of the fixed code shows average accuracy of 0.6**

In [440]:
# Doing 100 simulations of the fixed code:
snp_y = snp_matrix['PHENOTYPE'] - 1
only_snps = snp_matrix.drop(columns=['FID', 'IID', 'PAT', 'MAT', 'PHENOTYPE', 'SEX'])
only_snps = only_snps.fillna(0)

# Placeholder for results
accuracies = []
aucs = []

for i in range(100):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(only_snps, snp_y, test_size=0.2)
    
    # Reshape the data
    X_train = X_train.to_numpy().reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], X_test.shape[1], 1))
    input_shape = X_test.shape[1:]
    
    # Build the model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(rate=0.5))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(rate=0.5))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    sgd = SGD(learning_rate=0.01, momentum=0.08)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(X_train, y_train, batch_size=128, epochs=95, verbose=0)
    
    # Evaluate the model
    y_pred = model.predict(X_test).ravel()
    accuracy = accuracy_score(y_test, np.round(y_pred))
    auc = roc_auc_score(y_test, y_pred)
    
    # Store the results
    accuracies.append(accuracy)
    aucs.append(auc)

# Calculate mean and standard deviation
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
mean_auc = np.mean(aucs)
std_auc = np.std(aucs)

# Print the results
print(f'Average Accuracy: {mean_accuracy:.4f}, Std Dev Accuracy: {std_accuracy:.4f}')
print(f'Average AUC: {mean_auc:.4f}, Std Dev AUC: {std_auc:.4f}')

25/25 [==============================] - 0s 3ms/step
Average Accuracy: 0.5986, Std Dev Accuracy: 0.0197
Average AUC: 0.6056, Std Dev AUC: 0.0213
